In [1245]:
from constraint import *
from itertools import *

# Configuration and Setup

In [1246]:
# Configure day naming constants
M,T,W,TH,F = ['monday','tuesday','wednesday','thursday','friday']

In [1247]:
# Configure number of free hours each day
#
# Sample :
#   free_time[W] = 3  # Sets wednesday to 3 free hours

free_time = {}
free_time[M]  = 2  # Monday
free_time[T]  = 2  # Tuesday
free_time[W]  = 2  # Wednesday
free_time[TH] = 2  # Thursday
free_time[F]  = 1  # Friday

In [1248]:
# Configure tasks to complete this week
#
# Sample tasks
#   tasks['name'] = { 'effort': 1 }

tasks = {}
tasks['laundry'] = { 'effort': 2 }
tasks['pay bills'] = { 'effort': 2 }
tasks['wash the car'] = { 'effort': 3 }

# Precalculate intermediate values

In [1249]:
# Calculate helper variables
# Descriptions below
total_free_hours = 0
hour_to_day = {}

for day in [M,T,W,TH,F]:
    for hour in range(0,free_time[day]):
      hour_to_day[total_free_hours] = day
      total_free_hours += 1

hours_in_week = range(0,total_free_hours)

In [1250]:
# hour_to_day is a mapping of hour in week to a day
hour_to_day

{0: 'monday',
 1: 'monday',
 2: 'tuesday',
 3: 'tuesday',
 4: 'wednesday',
 5: 'wednesday',
 6: 'thursday',
 7: 'thursday',
 8: 'friday'}

In [1251]:
# total_free_hours is a sum of all the free hours this week 
total_free_hours

9

In [1252]:
# hours_in_week is an incremental array of hours in a week
hours_in_week

[0, 1, 2, 3, 4, 5, 6, 7, 8]

# Defining the constraint solver

In [1253]:
problem = Problem()
# task.keys() , i.e. ['laundry', 'pay bills', ...] are the variables
# hours_in_week, i.e. [0,1,2,3,...] are the possible values for the variables
# 
# The constraint solver will try assigning different hours of the week
# to the variables, and reject all those that do not meet the constraints
problem.addVariables(tasks.keys(),hours_in_week)

## Add Constraint: Two tasks cannot begin at the same time

In [1254]:
# Add a constraint so that no task begins on an identical hour to another task
problem.addConstraint(AllDifferentConstraint())

## Add Constraint: A task needs enough time to complaint before the week ends

In [1255]:
# Iterate over each task
# Add a constraint that validates the start time of a task allows enough time to complete it
for task_name in tasks.keys():
 problem.addConstraint(lambda start: start <= total_free_hours - tasks[task_name]['effort'], [task_name])

## Add Constraint: Two tasks can't be worked on at the same time

In [1275]:
for two_tasks in permutations(tasks.keys(),2):
    problem.addConstraint(lambda first_task, second_task: first_task + tasks[two_tasks[0]]['effort'] <= second_task if first_task < second_task else True, [two_tasks[0], two_tasks[1]])
    problem.addConstraint(lambda first_task, second_task: second_task + tasks[two_tasks[1]]['effort'] <= first_task if first_task > second_task else True, [two_tasks[0], two_tasks[1]])

# Get Solutions

In [1276]:
# Get a solution
solution = problem.getSolution()
solution

{'laundry': 7, 'pay bills': 1, 'wash the car': 4}

In [1277]:
number = problem.getSolutions()
len(number)

24

# Pretty Print Solution

In [1278]:
def print_schedule():
    prev_day = None
    for hour in hour_to_day.keys():
        # Print day
        if hour_to_day[hour] != prev_day :
            print "\n","== "+ hour_to_day[hour] + " =="
            prev_day = hour_to_day[hour]
        # Print task name
        current_task = None
        for task_name in solution:
            if (hour == solution[task_name]):
                if ( (current_task == None) or (solution[current_task] < solution[task_name]) ):
                    current_task = task_name
        if(current_task != None):
            print "start -> ",current_task, tasks[current_task]
        else:
            print "-"

In [1279]:
print_schedule()


== monday ==
-
start ->  pay bills {'effort': 2}

== tuesday ==
-
-

== wednesday ==
start ->  wash the car {'effort': 3}
-

== thursday ==
-
start ->  laundry {'effort': 2}

== friday ==
-
